In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import scipy as sp
import csv
import sympy
from torch.utils.data import DataLoader, TensorDataset
from qiskit import Aer, execute, QuantumCircuit
from qiskit.quantum_info import DensityMatrix, random_statevector
from IPython.display import clear_output

In [2]:
# Functions
def create_circuit():
    qc = QuantumCircuit(3)
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(1, 2)
    return qc

# Function to measure in different bases
def measure_basis(qc, basis):
    if basis == 'ZZZ':
        qc.measure_all()
    elif basis == 'XXX':
        qc.h([0, 1, 2])
        qc.measure_all()
    elif basis == 'YYY':
        qc.sdg([0, 1, 2])
        qc.h([0, 1, 2])
        qc.measure_all()
    return qc

# Function to simulate measurements
def simulate_measurements(qc):
    backend = Aer.get_backend('qasm_simulator')
    result = execute(qc, backend, shots=1024).result()
    counts = result.get_counts()
    return counts

# Function to collect measurement data
def collect_data(statevector):
    bases = ['ZZZ', 'XXX', 'YYY']
    data = []
    for basis in bases:
        qc = QuantumCircuit(3)
        qc.initialize(statevector, [0, 1, 2])
        qc = create_circuit().compose(qc)
        qc = measure_basis(qc, basis)
        counts = simulate_measurements(qc)
        data.append(counts)
    return data

# Convert measurement counts to probabilities
def counts_to_probabilities(counts, num_qubits):
    shots = sum(counts.values())
    probabilities = {k: v / shots for k, v in counts.items()}
    # Ensure all measurement outcomes are present
    for i in range(2 ** num_qubits):
        key = format(i, f'0{num_qubits}b')
        if key not in probabilities:
            probabilities[key] = 0.0
    return probabilities

# Preprocess the training data
def preprocess_data(training_data, num_qubits):
    processed_data = []
    for sample in training_data:
        sample_data = []
        for counts in sample:
            probabilities = counts_to_probabilities(counts, num_qubits)
            sample_data.extend([probabilities[format(i, f'0{num_qubits}b')] for i in range(2 ** num_qubits)])
        processed_data.append(sample_data)
    return np.array(processed_data)

In [6]:
# Import data from csv
processed_training_data = np.loadtxt('3q-processed_training_data_20000.csv', delimiter=',')
training_labels = np.loadtxt('3q-training_labels_20000.csv', delimiter=',')

# Convert to PyTorch tensors
X_train = torch.tensor(processed_training_data, dtype=torch.float32)
y_train = torch.tensor(training_labels, dtype=torch.float32)

In [7]:
# NN and Loss
class FidelityLoss(nn.Module):
    def forward(self, predicted, target):
        real_pred = predicted[:, :64].view(-1, 8, 8)
        imag_pred = predicted[:, 64:].view(-1, 8, 8)
        pred_density_matrix = real_pred + 1j * imag_pred
        real_target = target[:, :64].view(-1, 8, 8)
        imag_target = target[:, 64:].view(-1, 8, 8)
        target_density_matrix = real_target + 1j * imag_target
        
        # Compute the Frobenius norm between the predicted and target density matrices
        diff = pred_density_matrix - target_density_matrix
        frobenius_norm = torch.norm(diff, p='fro')
        
        loss = frobenius_norm.mean()
        return loss

class QuantumTomographyNN(nn.Module):
    def __init__(self):
        super(QuantumTomographyNN, self).__init__()
        self.fc1 = nn.Linear(24, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)
        self.fc6 = nn.Linear(64, 128)  # Adjusted output size for 3-qubit density matrix (64 real + 64 imaginary)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        x = self.dropout(x)
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = self.fc6(x)
        return x

In [8]:
# Initialize the model, loss function, and optimizer
model = QuantumTomographyNN()
criterion = FidelityLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

# Create DataLoader for batching
dataset = TensorDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 200
for epoch in range(num_epochs):
    running_loss = 0.0
    for batch_X, batch_y in dataloader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        running_loss += loss.item()
    scheduler.step()
    if (epoch + 1) % 10 == 0:
        avg_loss = running_loss / len(dataloader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

Epoch [10/200], Loss: 4.9513
Epoch [20/200], Loss: 4.9145
Epoch [30/200], Loss: 4.8961
Epoch [40/200], Loss: 4.8962
Epoch [50/200], Loss: 4.8980
Epoch [60/200], Loss: 4.7550
Epoch [70/200], Loss: 4.7372
Epoch [80/200], Loss: 4.7218
Epoch [90/200], Loss: 4.7165
Epoch [100/200], Loss: 4.7111
Epoch [110/200], Loss: 4.6969
Epoch [120/200], Loss: 4.6942
Epoch [130/200], Loss: 4.6911
Epoch [140/200], Loss: 4.6911
Epoch [150/200], Loss: 4.6910
Epoch [160/200], Loss: 4.6889
Epoch [170/200], Loss: 4.6884
Epoch [180/200], Loss: 4.6898
Epoch [190/200], Loss: 4.6885
Epoch [200/200], Loss: 4.6878


In [9]:
def reconstruct_density_matrix(measurement_data):
    processed_data = preprocess_data([measurement_data], num_qubits)
    input_tensor = torch.tensor(processed_data, dtype=torch.float32)
    with torch.no_grad():
        predicted_elements = model(input_tensor).numpy()
    real_part = predicted_elements[:, :64]
    imag_part = predicted_elements[:, 64:]
    combined_matrix = real_part + 1j * imag_part
    return combined_matrix.reshape((8, 8))

def check_fidelity(rho, sigma):
    fidelity = (np.trace(sp.linalg.sqrtm(np.matmul(rho, sigma))))**2
    return np.real(fidelity)

def avrg_fidelity(n):
    fids = []
    for i in range(n):
        statevector = random_statevector(2**num_qubits).data
        new_data = collect_data(statevector)
        reconstructed_density_matrix = reconstruct_density_matrix(new_data)
        qc = QuantumCircuit(3)
        qc.initialize(statevector, [0, 1, 2])
        qc = create_circuit().compose(qc)
        state = execute(qc, Aer.get_backend('statevector_simulator')).result().get_statevector()
        original_density_matrix = DensityMatrix(state).data
        fid = check_fidelity(reconstructed_density_matrix, original_density_matrix)
        print(fid)
        fids.append(fid)

        clear_output(wait=True)
        print(f"Finished with iteration number: {i}")
    print(f"mean: {np.mean(fids)}")
    print(f"std: {np.std(fids)}")
    return sum(fids) / n

In [10]:
num_qubits = 3
avrg_fidelity(1000)

Finished with iteration number: 999
mean: 0.31261507128094285
std: 0.07008680846490746


0.3126150712809428

In [115]:
#Example of reconstructed state
statevector = random_statevector(2**num_qubits).data
new_data = collect_data(statevector)
reconstructed_density_matrix = reconstruct_density_matrix(new_data)
qc = QuantumCircuit(2)
qc.initialize(statevector, [0, 1])
qc = create_circuit().compose(qc)
state = execute(qc, Aer.get_backend('statevector_simulator')).result().get_statevector()
original_density_matrix = DensityMatrix(state).data
clear_output()

In [116]:
M = sympy.Matrix(reconstructed_density_matrix)
M.applyfunc(lambda x: round(x, 3))

Matrix([
[           0.24, 0.006 - 0.068*I, 0.121 - 0.208*I,  0.133 - 0.21*I],
[0.006 + 0.068*I,           0.083, 0.212 - 0.037*I, 0.033 - 0.003*I],
[0.121 + 0.208*I, 0.212 + 0.037*I,           0.395,  0.261 - 0.07*I],
[ 0.133 + 0.21*I, 0.033 + 0.003*I,  0.261 + 0.07*I,           0.282]])

In [117]:
M = sympy.Matrix(original_density_matrix)
M.applyfunc(lambda x: round(x, 3))

Matrix([
[           0.237, 0.014 - 0.187*I, -0.008 - 0.274*I, 0.096 - 0.248*I],
[ 0.014 + 0.187*I,           0.149,  0.216 - 0.023*I,  0.201 + 0.06*I],
[-0.008 + 0.274*I, 0.216 + 0.023*I,            0.317, 0.283 + 0.118*I],
[ 0.096 + 0.248*I,  0.201 - 0.06*I,  0.283 - 0.118*I,           0.297]])

In [118]:
check_fidelity(original_density_matrix, reconstructed_density_matrix)

0.7833189200989189